In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

In [ ]:
df = pd.read_csv('BTC_USD_2018-01-01_2020-12-31-CoinDesk.csv')

currency = df[['Currency']]
date = df[['Date']]

df = df.drop(columns=['Currency', 'Date'])

df.head()

,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,13439.41750,13062.14500,14213.44125,12587.60375
1,13337.62125,13439.41750,13892.24250,12859.80250
2,14881.54500,13337.62125,15216.75625,12955.96500
3,15104.45000,14881.54500,15394.98625,14588.59500
4,14953.85250,15104.45000,15194.40625,14225.16625


In [ ]:
df.describe()

,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
count,1096.000000,1096.000000,1096.000000,1096.000000
mean,8651.325055,8636.463533,8862.612775,8399.638971
std,3615.210632,3568.811796,3697.290208,3470.183975
min,3194.962272,3178.299355,3227.867420,3122.342716
25%,6471.303804,6469.304513,6621.041482,6356.728946
50%,8277.820672,8277.819507,8510.579808,8061.897433
75%,10169.090147,10175.577984,10386.723547,9820.884326
max,28768.836208,27349.327233,28928.214391,27349.283204


In [ ]:
# Mengidentifikasi dan menangani outliers menggunakan IQR
def handle_outliers_iqr(df):
    # Membuat salinan DataFrame asli untuk diolah
    df_iqr = df.copy()
    outliers_count = 0

    for column in df_iqr.columns:
        Q1 = df_iqr[column].quantile(0.25)
        Q3 = df_iqr[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Menghitung jumlah outliers
        outliers_count += df_iqr[(df_iqr[column] < lower_bound) | (df_iqr[column] > upper_bound)].shape[0]

        # Mengganti outliers dengan NaN
        df_iqr[column] = np.where((df_iqr[column] < lower_bound) | (df_iqr[column] > upper_bound), np.nan, df_iqr[column])

    return df_iqr, outliers_count

# Mengganti outliers dengan NaN menggunakan IQR
df_with_nan_iqr, outliers_count = handle_outliers_iqr(df)
print("Jumlah outliers yang terdeteksi:", outliers_count)

# Mengganti outliers dengan nilai median kolom menggunakan IQR
df_with_median_iqr = df_with_nan_iqr.copy()
for column in df_with_median_iqr.columns:
    median = df_with_median_iqr[column].median()
    df_with_median_iqr[column].fillna(median, inplace=True)
print("Jumlah outliers yang diganti dengan median:", df_with_nan_iqr.isna().sum().sum())

# Mengganti outliers dengan nilai mean kolom menggunakan IQR
df_with_mean_iqr = df_with_nan_iqr.copy()
for column in df_with_mean_iqr.columns:
    mean = df_with_mean_iqr[column].mean()
    df_with_mean_iqr[column].fillna(mean, inplace=True)
print("Jumlah outliers yang diganti dengan mean:", df_with_nan_iqr.isna().sum().sum())

# Menghapus baris yang mengandung outliers menggunakan IQR
df_without_outliers_iqr = df_with_nan_iqr.dropna()
print("Jumlah baris yang tersisa setelah menghapus outliers:", df_without_outliers_iqr.shape[0])

Jumlah outliers yang terdeteksi: 208
Jumlah outliers yang diganti dengan median: 208
Jumlah outliers yang diganti dengan mean: 208
Jumlah baris yang tersisa setelah menghapus outliers: 1041


In [ ]:
df_without_outliers_iqr['Currency'] = currency
df_without_outliers_iqr['Date'] = date

C:\Users\ACER\AppData\Local\Temp\ipykernel_12680\1150556890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_outliers_iqr['Currency'] = currency
C:\Users\ACER\AppData\Local\Temp\ipykernel_12680\1150556890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_outliers_iqr['Date'] = date


In [ ]:
# Memastikan 'Currency' adalah kolom pertama
cols = list(df_without_outliers_iqr.columns)
cols.remove('Date')  # Menghapus 'Currency' dari list
cols.insert(0, 'Date')  # Menambahkan 'Currency' di posisi pertama
cols.remove('Currency')
cols.insert(0, 'Currency')

In [ ]:
# Membuat DataFrame baru dengan kolom 'Currency' di depan
df_new = df_without_outliers_iqr[cols]

df_new.describe()

,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
count,1041.000000,1041.000000,1041.000000,1041.000000
mean,8067.167317,8064.528702,8268.586665,7839.748354
std,2525.125546,2520.686660,2597.556532,2431.400756
min,3194.962272,3178.299355,3227.867420,3122.342716
25%,6405.500726,6405.565596,6544.614891,6319.713209
50%,8151.788680,8159.256250,8331.291686,7884.518625
75%,9737.458559,9737.457447,9962.180331,9521.832102
max,15500.334255,15632.411250,15944.522500,14845.791762


In [ ]:
df_new.to_csv('IQR.csv', index=False)